In [1]:
import numpy as np
from fastai import *
from fastai.vision import *
import torch
import os
from torchvision import transforms
import torch.nn.functional as F

**data from https://www.kaggle.com/mloey1/ahcd1**

In [2]:
data_path = Path('data')

In [3]:
test_x = np.genfromtxt(data_path/'csvTestImages 3360x1024.csv', delimiter=',')
test_y = np.genfromtxt(data_path/'csvTestLabel 3360x1.csv', delimiter=',')
test_y = test_y.astype(int) - 1
test_x = test_x.astype(int)


test_x.shape, test_y.shape, test_x[0].shape, test_y.max(), test_y.min(), max(test_x.max(axis=1))

((3360, 1024), (3360,), (1024,), 27, 0, 255)

#### Custom Dataset

In [4]:
class ArabicMnist(Dataset):
    def __init__(self, X, y=None):
        super().__init__()
        self.classes = np.unique(y)
        self.c = len(np.unique(y))
        self.X = X
        if y is not None: self.y = y

    def __getitem__(self, i):
        image = self.X[i]
        arr3D = np.tile(image[..., None], 3)
        arr3D = np.transpose(arr3D, (2, 1, 0))
        new_tensor = torch.from_numpy(arr3D).float()
        return (new_tensor, self.y[i])
    
    def __len__(self): return len(self.X)

In [5]:
test_x_reshaped = test_x.reshape([-1, 32, 32])

In [6]:
AM = ArabicMnist(test_x_reshaped[:3000], test_y[:3000])
AMV = ArabicMnist(test_x_reshaped[3000:], test_y[3000:])

In [7]:
data = DataBunch.create(AM, AMV, bs=128)

#### Loss here will be negative

In [8]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [9]:
learn.loss_func

<function torch.nn.functional.nll_loss(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')>

In [10]:
# %%debug
learn.fit(3, 1e-2)

Total time: 00:07
epoch  train_loss  valid_loss  error_rate  
1      -12.764399  -29.064812  0.719444      (00:04)
2      -62.974148  -157.600830  0.588889     (00:01)
3      -172.593765  -409.579071  0.416667    (00:01)



#### Setup the proper loss function and run again

In [11]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [12]:
learn.loss_func

<function torch.nn.functional.nll_loss(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')>

In [13]:
learn.loss_func = F.cross_entropy

In [14]:
learn.loss_func

<function torch.nn.functional.cross_entropy(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')>

In [15]:
# %%debug
learn.fit(3, 1e-2)

Total time: 00:05
epoch  train_loss  valid_loss  error_rate
1      2.545666    2.053045    0.650000    (00:01)
2      1.797734    1.070664    0.336111    (00:01)
3      1.360770    0.984398    0.308333    (00:01)



### MNSIT Fashion

In [ ]:
train_df = pd.read_csv(str(data_path / 'fashion-mnist_train.csv'))
train_df.head()

In [ ]:
number = 0
images, labels = [], []
def convert_to_numpy(row):
    global number
    label = row['label']
    pixels = []
    if number > 0:
        for i in range(1, 785):
            pixels.append(int(row['pixel' + str(i)]))
        pixels = np.array(pixels).reshape(28, 28)
        images.append(pixels)
        labels.append(label)
    number += 1
    return row

In [ ]:
_ = train_df.apply(convert_to_numpy, axis=1)

In [ ]:
len(images), len(labels)

In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
images.shape, labels.shape

In [ ]:
AM = ArabicMnist(images[:30000], labels[:30000])
AMV = ArabicMnist(images[30000:], labels[30000:])

In [ ]:
data = DataBunch.create(AM, AMV, bs=64)

In [ ]:
data.c

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [ ]:
learn.loss_func

In [ ]:
# %%debug
learn.fit(3, 1e-9)